# 01/20

%matplotlib inline

In [1]:
description = 'mobilenet_from_pretrain_emotion'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_20_22:19:05.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
mobilenet_pretrained = tf.keras.models.load_model('../models/affectnet_emotions/mobilenet_7.h5')

In [8]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [9]:
hidden_layers = [256]

In [10]:
"""valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat)

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat)
"""
emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_emotion) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)       0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                   

                                                                 
 conv_dw_13_relu (ReLU)      (None, 7, 7, 1024)        0         
                                                                 
 conv_pw_13 (Conv2D)         (None, 7, 7, 1024)        1048576   
                                                                 
 conv_pw_13_bn (BatchNormali  (None, 7, 7, 1024)       4096      
 zation)                                                         
                                                                 
 conv_pw_13_relu (ReLU)      (None, 7, 7, 1024)        0         
                                                                 
 global_pooling (GlobalAvera  (None, 1024)             0         
 gePooling2D)                                                    
                                                                 
 feat_emotion (Dense)        (None, 256)               262400    
                                                                 
 outputs_e

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [14]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_emotion = logits

            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        #valence_loss = float(valence_loss)
        #arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        #total_valence_loss += valence_loss
        #total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            #'valence_loss': valence_loss,
            #'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
        """if step % 10 == 0:
            lprint(
                f"Training loss at epoch {epoch} step {step}: {total_loss/total_step}\n",
                f"This round's {valence_loss=}, {arousal_loss=}, {emotion_loss=}"
            )
            lprint("Seen so far: %s samples" % ((step + 1) * batch_size))
            lprint(f"--- {time.time() - start_time} seconds for iter {total_step} step, each step have {batch_size}, so the model train with {total_step*batch_size} img this iter ---\n")
            total_loss = 0
            total_step = 0

            start_time = time.time()"""
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        #'total_valence_loss': total_valence_loss,
        #'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 0, "emotion_loss": 2.7221484184265137, "time_take": 5.099516153335571}
{"emotion_correct": 10, "emotion_loss": 2.117711305618286, "time_take": 5.558592319488525}
{"emotion_correct": 16, "emotion_loss": 1.7100034952163696, "time_take": 5.9020912647247314}
{"emotion_correct": 20, "emotion_loss": 1.6344716548919678, "time_take": 6.357923746109009}
{"emotion_correct": 18, "emotion_loss": 1.3840745687484741, "time_take": 6.828293085098267}
{"emotion_correct": 18, "emotion_loss": 1.4440670013427734, "time_take": 7.209093809127808}
{"emotion_correct": 18, "emotion_loss": 1.5268759727478027, "time_take": 7.623162269592285}
{"emotion_correct": 21, "emotion_loss": 1.1724348068237305, "time_take": 8.183065414428711}
{"emotion_correct": 18, "emotion_loss": 1.4057260751724243, "time_take": 8.608858585357666}
{"emotion_correct": 18, "emotion_loss": 1.6142172813415527, "time_take": 8.93387746810913}
{"emotion_correct": 23, "emotion_loss": 0.8827458620071411, "time_take": 9.4275910

{"emotion_correct": 18, "emotion_loss": 1.650761365890503, "time_take": 36.602052450180054}
{"emotion_correct": 20, "emotion_loss": 1.0479695796966553, "time_take": 36.86468529701233}
{"emotion_correct": 19, "emotion_loss": 1.2855257987976074, "time_take": 37.22829985618591}
{"emotion_correct": 14, "emotion_loss": 1.769044041633606, "time_take": 37.6101758480072}
{"emotion_correct": 20, "emotion_loss": 1.4007195234298706, "time_take": 37.89427709579468}
{"emotion_correct": 18, "emotion_loss": 1.5334444046020508, "time_take": 38.22890901565552}
{"emotion_correct": 18, "emotion_loss": 1.2629543542861938, "time_take": 38.52359318733215}
{"emotion_correct": 17, "emotion_loss": 1.2916805744171143, "time_take": 38.85490274429321}
{"emotion_correct": 18, "emotion_loss": 1.146187663078308, "time_take": 39.19103693962097}
{"emotion_correct": 15, "emotion_loss": 1.605285406112671, "time_take": 39.53837275505066}
{"emotion_correct": 17, "emotion_loss": 1.2171475887298584, "time_take": 39.91680407

{"emotion_correct": 17, "emotion_loss": 1.4488705396652222, "time_take": 65.34416079521179}
{"emotion_correct": 20, "emotion_loss": 1.3931961059570312, "time_take": 65.71899032592773}
{"emotion_correct": 20, "emotion_loss": 1.1829814910888672, "time_take": 66.0728485584259}
{"emotion_correct": 12, "emotion_loss": 1.4593939781188965, "time_take": 66.36272525787354}
{"emotion_correct": 20, "emotion_loss": 1.2473537921905518, "time_take": 66.71180510520935}
{"emotion_correct": 19, "emotion_loss": 1.2592203617095947, "time_take": 67.1260404586792}
{"emotion_correct": 20, "emotion_loss": 1.035043716430664, "time_take": 67.44825553894043}
{"emotion_correct": 16, "emotion_loss": 1.6011412143707275, "time_take": 67.76746940612793}
{"emotion_correct": 19, "emotion_loss": 1.3205173015594482, "time_take": 68.20105075836182}
{"emotion_correct": 15, "emotion_loss": 1.5702848434448242, "time_take": 68.97315287590027}
{"emotion_correct": 25, "emotion_loss": 0.8833320140838623, "time_take": 69.2792892

{"emotion_correct": 18, "emotion_loss": 1.3042715787887573, "time_take": 95.07454657554626}
{"emotion_correct": 17, "emotion_loss": 1.364998698234558, "time_take": 95.31759524345398}
{"emotion_correct": 12, "emotion_loss": 1.4367235898971558, "time_take": 95.55460095405579}
{"emotion_correct": 15, "emotion_loss": 1.2921693325042725, "time_take": 95.86115455627441}
{"emotion_correct": 21, "emotion_loss": 1.258453369140625, "time_take": 96.19877648353577}
{"emotion_correct": 20, "emotion_loss": 0.8913072347640991, "time_take": 96.47846746444702}
{"emotion_correct": 22, "emotion_loss": 1.1229816675186157, "time_take": 96.7984790802002}
{"emotion_correct": 16, "emotion_loss": 1.4646599292755127, "time_take": 97.15815758705139}
{"emotion_correct": 21, "emotion_loss": 1.1677534580230713, "time_take": 97.45276260375977}
{"emotion_correct": 16, "emotion_loss": 1.5721335411071777, "time_take": 97.89337062835693}
{"emotion_correct": 15, "emotion_loss": 1.475423812866211, "time_take": 98.17646980

{"emotion_correct": 17, "emotion_loss": 1.4385257959365845, "time_take": 127.83462452888489}
{"emotion_correct": 23, "emotion_loss": 1.0892457962036133, "time_take": 128.0784397125244}
{"emotion_correct": 16, "emotion_loss": 1.343300700187683, "time_take": 128.43666696548462}
{"emotion_correct": 15, "emotion_loss": 1.6263468265533447, "time_take": 128.99571561813354}
{"emotion_correct": 19, "emotion_loss": 1.1764719486236572, "time_take": 129.35964107513428}
{"emotion_correct": 20, "emotion_loss": 1.2207252979278564, "time_take": 129.6472225189209}
{"emotion_correct": 17, "emotion_loss": 1.3808969259262085, "time_take": 129.93035221099854}
{"emotion_correct": 14, "emotion_loss": 1.5648860931396484, "time_take": 130.4216763973236}
{"emotion_correct": 19, "emotion_loss": 1.365675926208496, "time_take": 130.78464436531067}
{"emotion_correct": 19, "emotion_loss": 1.2995235919952393, "time_take": 131.07791447639465}
{"emotion_correct": 16, "emotion_loss": 1.4903017282485962, "time_take": 13

...

{"emotion_correct": 17, "emotion_loss": 1.3234877586364746, "time_take": 954.2154588699341}
{"emotion_correct": 19, "emotion_loss": 1.120121955871582, "time_take": 954.5570659637451}
{"emotion_correct": 15, "emotion_loss": 1.4850077629089355, "time_take": 955.0594136714935}
{"emotion_correct": 20, "emotion_loss": 1.1745022535324097, "time_take": 955.5282053947449}
{"emotion_correct": 21, "emotion_loss": 0.8244796991348267, "time_take": 955.9551923274994}
{"emotion_correct": 17, "emotion_loss": 1.3657915592193604, "time_take": 956.2985897064209}
{"emotion_correct": 22, "emotion_loss": 1.0301153659820557, "time_take": 956.5386319160461}
{"emotion_correct": 20, "emotion_loss": 1.2831817865371704, "time_take": 956.9657146930695}
{"emotion_correct": 20, "emotion_loss": 1.1047543287277222, "time_take": 957.3825833797455}
{"emotion_correct": 20, "emotion_loss": 1.1493728160858154, "time_take": 957.8318877220154}
{"emotion_correct": 18, "emotion_loss": 1.0270397663116455, "time_take": 958.1957

{"emotion_correct": 19, "emotion_loss": 1.443608045578003, "time_take": 989.3546087741852}
{"emotion_correct": 19, "emotion_loss": 1.1426254510879517, "time_take": 989.7145421504974}
{"emotion_correct": 22, "emotion_loss": 1.1076644659042358, "time_take": 990.2420976161957}
{"emotion_correct": 22, "emotion_loss": 1.033632755279541, "time_take": 990.4841349124908}
{"emotion_correct": 24, "emotion_loss": 0.7051866054534912, "time_take": 990.8246002197266}
{"emotion_correct": 21, "emotion_loss": 1.0739117860794067, "time_take": 991.2090773582458}
{"emotion_correct": 18, "emotion_loss": 1.226359486579895, "time_take": 991.6353452205658}
{"emotion_correct": 23, "emotion_loss": 1.1148850917816162, "time_take": 992.144855260849}
{"emotion_correct": 17, "emotion_loss": 1.5083773136138916, "time_take": 992.5827331542969}
{"emotion_correct": 21, "emotion_loss": 1.0772602558135986, "time_take": 992.9327788352966}
{"emotion_correct": 18, "emotion_loss": 1.3314056396484375, "time_take": 993.3616983

{"emotion_correct": 22, "emotion_loss": 0.9923292994499207, "time_take": 1023.8320534229279}
{"emotion_correct": 22, "emotion_loss": 1.0436376333236694, "time_take": 1024.2250232696533}
{"emotion_correct": 20, "emotion_loss": 1.4239320755004883, "time_take": 1024.6677389144897}
{"emotion_correct": 17, "emotion_loss": 1.3765583038330078, "time_take": 1024.955545425415}
{"emotion_correct": 21, "emotion_loss": 0.8741892576217651, "time_take": 1025.3433556556702}
{"emotion_correct": 15, "emotion_loss": 1.4456982612609863, "time_take": 1025.6777217388153}
{"emotion_correct": 20, "emotion_loss": 1.0934643745422363, "time_take": 1025.9451158046722}
{"emotion_correct": 18, "emotion_loss": 1.4640913009643555, "time_take": 1026.2698693275452}
{"emotion_correct": 17, "emotion_loss": 1.4361215829849243, "time_take": 1026.6354460716248}
{"emotion_correct": 22, "emotion_loss": 0.9436010122299194, "time_take": 1026.9601438045502}
{"emotion_correct": 20, "emotion_loss": 1.2127854824066162, "time_take"

{"emotion_correct": 20, "emotion_loss": 1.0618956089019775, "time_take": 1056.3852133750916}
{"emotion_correct": 20, "emotion_loss": 1.0682380199432373, "time_take": 1056.795418024063}
{"emotion_correct": 20, "emotion_loss": 1.1264744997024536, "time_take": 1057.2316138744354}
{"emotion_correct": 19, "emotion_loss": 1.1967734098434448, "time_take": 1057.529619693756}
{"emotion_correct": 21, "emotion_loss": 1.1866480112075806, "time_take": 1058.1323795318604}
{"emotion_correct": 18, "emotion_loss": 1.2196471691131592, "time_take": 1058.4868369102478}
{"emotion_correct": 19, "emotion_loss": 1.0503475666046143, "time_take": 1058.9649465084076}
{"emotion_correct": 17, "emotion_loss": 1.5933239459991455, "time_take": 1059.3318259716034}
{"emotion_correct": 22, "emotion_loss": 0.9206979870796204, "time_take": 1059.7163560390472}
{"emotion_correct": 16, "emotion_loss": 1.3601605892181396, "time_take": 1060.553039073944}
{"emotion_correct": 18, "emotion_loss": 1.3805935382843018, "time_take": 

{"emotion_correct": 20, "emotion_loss": 1.306304931640625, "time_take": 1091.1430275440216}
{"emotion_correct": 19, "emotion_loss": 1.1184194087982178, "time_take": 1091.3899464607239}
{"emotion_correct": 19, "emotion_loss": 1.2106215953826904, "time_take": 1091.8100745677948}
{"emotion_correct": 18, "emotion_loss": 1.319722056388855, "time_take": 1092.1422233581543}
{"emotion_correct": 20, "emotion_loss": 1.367675542831421, "time_take": 1092.545164823532}
{"emotion_correct": 17, "emotion_loss": 1.6322113275527954, "time_take": 1092.98881483078}
{"emotion_correct": 21, "emotion_loss": 1.1815383434295654, "time_take": 1093.4029734134674}
{"emotion_correct": 20, "emotion_loss": 1.1206990480422974, "time_take": 1093.8913292884827}
{"emotion_correct": 18, "emotion_loss": 1.3069908618927002, "time_take": 1094.170926809311}
{"emotion_correct": 20, "emotion_loss": 1.1812691688537598, "time_take": 1094.7065904140472}
{"emotion_correct": 13, "emotion_loss": 1.5104764699935913, "time_take": 1094